In [320]:
from openai import OpenAI;
from dotenv import load_dotenv;
from pypdf import PdfReader;
import json;
import requests;
import gradio as gr;
import os;


In [321]:
load_dotenv(override=True);

open_ai_api = os.getenv("OPENAI_API_KEY");
pushover_user = os.getenv("PUSHOVER_USER");
pushover_token = os.getenv("PUSHOVER_TOKEN");

openai = OpenAI()


In [322]:
def push(message):
    return requests.post(
        "https://api.pushover.net/1/messages.json",
        data = {"user": pushover_user, "token": pushover_token, "message": message}
    )

In [323]:
push("hey")

<Response [200]>

In [324]:
def record_user_details(email, name, message="not provides"):
    try:
        push(f"New user details received: \nEmail: {email}\nName: {name}\nMessage: {message}");
    except Exception as e:
        print('API call failed', e)
    return {"recorded": "ok"}


In [325]:
def record_unknown_question(question):
    try:
        push(f"New unknown question received: \nQuestion: {question}");
    except Exception as e:
        print('API call failed', e)
    return {"recorded": "ok"}


In [326]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "message": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {"question": {"type": "string", "description": "The question that couldn't be answered"}}
    },
    "required": ["question"],
    "additionalProperties": False
}

In [327]:
tools = [
    {"type": "function", "function": record_user_details_json},
    {"type": "function", "function": record_unknown_question_json}
]


In [328]:
def handle_tool_call(tool_calls):
    results = [];
    for tool_call in tool_calls:
        tool_name = tool_call.function.name;
        args = json.loads(tool_call.function.arguments);
        if tool_name == "record_user_details":
            result = record_user_details(**args)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**args)
        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id});
    return results

In [329]:
linkedin_pdf_reader = PdfReader("../me/linkedin.pdf");
resume_pdf_reader = PdfReader("../me/resume.pdf");

In [330]:
linkedin = ""
for page in linkedin_pdf_reader.pages:
    text = page.extract_text();
    if text:
        linkedin += text;

resume = ""
for page in resume_pdf_reader.pages:
    text = page.extract_text();
    if text:
        resume += text;



In [331]:
with open("../me/summary.txt", "r") as file:
    summary = file.read();


In [332]:
name = "Krishna Sridhar";


In [333]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n ## Resume:\n{resume}"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [334]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [335]:
evaluator_system_prompt = f"""You are an evaluator for a {name}'s website chatbot agent. \
    You are given a conversation between a user and an agent. \
    Your task is to decide whether the Agent's latest response is acceptable quality and determine if it is in character as {name}. \
    You should look for the following: \
    - The chatbot's response is in character as {name}. \
    - The chatbot's response is professional and engaging. \
    - The chatbot's response is helpful and informative. \

    Here is the information:
    Summary: {summary}
    LinkedIn Profile: {linkedin}
    Resume: {resume}"""



In [336]:
def evaluator_user_prompt(reply, message, history):
    return f"""
    Here is the conversation:
    {history}
    Here is the latest message:
    {message}
    Here is the chatbot's recent message:
    {reply}"""

In [337]:
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [338]:
def evaluator(message, reply, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed


In [339]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message

In [340]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}];
    done = False;
    reply = ""
    while not done:
        client = OpenAI(api_key=open_ai_api);
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools)
        
        finish_reason = response.choices[0].finish_reason;
        reply = response.choices[0].message;
        print(reply.content, 'res 1')
        if finish_reason == "tool_calls":
            tool_calls = reply.tool_calls;
            results = handle_tool_call(tool_calls);
            messages.append(message);
            messages.extend(results);
        else:
            evaluation = evaluator(reply, message, history)
            print(evaluation)
            if(evaluation.is_acceptable):
                done = True
            else:
                reply = rerun(reply, message, history, evaluation.feedback)
                print(reply.content, 'res 2')
                done = True
            
    return reply.content;

    

In [341]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.


I'm glad to hear that you're interested in connecting! Please share your email address, and I’ll make sure to reach out. If you have any specific topics you'd like to discuss, feel free to mention those as well! res 1
is_acceptable=True feedback='The chatbot is very helpful and inviting.'
None res 1


Traceback (most recent call last):
  File "c:\Users\krish\agentic_ai\.venv\Lib\site-packages\gradio\queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\krish\agentic_ai\.venv\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\krish\agentic_ai\.venv\Lib\site-packages\gradio\blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\krish\agentic_ai\.venv\Lib\site-packages\gradio\blocks.py", line 1621, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\krish\agentic_ai\.venv\Lib\site-packages\gradio\utils.py", line 882, in async_wrapper
    response = await f(*args, **kwargs)
               ^^^^^^^^^^^^^^